In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import Image, SVG
import numpy as np
np.random.seed(0)


import os
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import (
    VGG19, 
    preprocess_input, 
    decode_predictions
)

In [2]:
#Load the model VGG19
model = VGG19(include_top=True, weights='imagenet')

In [3]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
model.layers

In [5]:
old_model_hidden_layer_output = model.layers[-4].output
my_new_layer_0 = Dense(2056, activation='elu', kernel_initializer='he_normal', name='my_fc1')(old_model_hidden_layer_output)
my_new_layer_1 = Dense(1024, activation='elu', kernel_initializer='he_normal', name='my_fc2')(my_new_layer_0)
my_new_layer_2 = Dense(512, activation='elu', kernel_initializer='he_normal', name='my_fc3')(my_new_layer_1)
my_new_layer_3 = Dense(4, activation='softmax', name='house_predictions')(my_new_layer_2)
my_new_model = Model(inputs=model.inputs, outputs=[my_new_layer_3])

In [6]:
my_new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
ROOT_PATH = os.path.join("data")
IMAGE_SIZE = (224, 224)

In [8]:
from typing import List, Tuple, Callable
from collections import namedtuple

In [9]:
Home = namedtuple("Home", ['style', 'number'])

In [10]:
HOMES = [Home('spanish', 700), Home('tudor', 755), Home('victorian', 853), Home('craftsman', 822)]

In [11]:
def get_one_image_of_style(style: str, image_number: int):
  image_path = os.path.join(ROOT_PATH, style, f"{style}_{image_number}.jpg")
  img = image.load_img(image_path, target_size=IMAGE_SIZE)
  return image.img_to_array(img)

assert get_one_image_of_style('tudor', 0).shape == (224, 224, 3)

In [12]:
def get_all_inputs_for_one_style(style: str, number_of_images: int):
  return np.array([get_one_image_of_style(style, image_number) for image_number in range(number_of_images)])

assert get_all_inputs_for_one_style('tudor', 10).shape[0] == 10

In [13]:
def get_outputs_for_one_style(homes_style: str, number_of_homes: int):
  return np.array([homes_style for _ in range(number_of_homes)])

assert not isinstance(get_outputs_for_one_style('generic', 2), list)
assert isinstance(get_outputs_for_one_style('generic', 2), np.ndarray)
assert list(get_outputs_for_one_style('generic', 2)) == ['generic', 'generic']

In [14]:
def merge_styles(homes: List[Home], func: Callable):
  all_styles = [func(home.style, home.number) for home in homes]
  return np.concatenate(all_styles, axis=0)

In [15]:
X = merge_styles(HOMES, get_all_inputs_for_one_style)
assert X.shape == (3130, 224, 224, 3)

In [16]:
y = merge_styles(HOMES, get_outputs_for_one_style)
assert y.shape == (3130,)

In [17]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,  random_state=0)

In [18]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train = label_encoder.transform(y_train)
y_train = to_categorical(y_train)
y_train.shape

(2347, 4)

In [19]:
y_test = to_categorical(label_encoder.transform(y_test))

In [20]:
y_test.shape

(783, 4)

In [21]:
opt = keras.optimizers.Nadam(learning_rate=.00002)

In [22]:
# Use categorical crossentropy for categorical data and mean squared error for regression
# Hint: your output layer in this example is using software for logistic regression (categorical)
# If your output layer activation was `linear` then you may want to use `mse` for loss
my_new_model.compile(optimizer=opt,
                     loss='categorical_crossentropy',
                     metrics=['accuracy', 'categorical_accuracy'])

In [ ]:
history = my_new_model.fit(X_train,
                           y_train,
                           batch_size=16,
                           epochs=5,
                           shuffle=True,
                           validation_split=.1)

Train on 2112 samples, validate on 235 samples
Epoch 1/5
2112/2112 [==============================] - 2337s 1s/sample - loss: 1.5762 - accuracy: 0.5431 - categorical_accuracy: 0.5431 - val_loss: 0.8128 - val_accuracy: 0.6851 - val_categorical_accuracy: 0.6851
Epoch 2/5
2112/2112 [==============================] - 2183s 1s/sample - loss: 0.4084 - accuracy: 0.8584 - categorical_accuracy: 0.8584 - val_loss: 0.7475 - val_accuracy: 0.7191 - val_categorical_accuracy: 0.7191
Epoch 3/5
1904/2112 [==========================>...] - ETA: 3:29 - loss: 0.1308 - accuracy: 0.9664 - categorical_accuracy: 0.9664

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
my_new_model.evaluate(x=X_test, y=y_test)